<a href="https://colab.research.google.com/github/muttu2244/capstoneproject_DriverDemandPrediction/blob/main/DriverDemandPredictionforOptimalFoodDeliveryCharges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.impute import SimpleImputer,MissingIndicator, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate,GridSearchCV,RepeatedStratifiedKFold,cross_val_score,cross_val_predict
from sklearn.preprocessing import PowerTransformer,OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer,make_column_selector,ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, GradientBoostingRegressor,RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_auc_score
from sklearn.impute import  KNNImputer
from sklearn.svm import SVC
from lightgbm import LGBMRegressor, LGBMClassifier
import xgboost as xgb

import warnings; warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
#from kuma_utils.preprocessing.imputer import LGBMImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
import holidays
import gc
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import SplineTransformer
from sklearn.preprocessing import PolynomialFeatures
from geopy.distance import geodesic

import gc


#from category_encoders import OrdinalEncoder


import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import KBinsDiscretizer,LabelEncoder

#from feature_engine.encoding import OrdinalEncoder
#from feature_engine.discretisation import EqualFrequencyDiscretiser
#from feature_engine.encoding import RareLabelEncoder
#from feature_engine.selection import DropDuplicateFeatures, DropConstantFeatures
#from feature_engine.selection import (RecursiveFeatureElimination, RecursiveFeatureAddition,
#    DropConstantFeatures,
#    DropDuplicateFeatures,
#    SmartCorrelatedSelection,DropCorrelatedFeatures
#)
#from feature_engine.creation import MathFeatures,RelativeFeatures

# to select the features
#from sklearn.feature_selection import SelectKBest, SelectPercentile
#from feature_engine.encoding import CountFrequencyEncoder
#from feature_engine.encoding import OneHotEncoder
#from feature_engine.encoding import MeanEncoder
#from sklearn import metrics
#from feature_engine.encoding import CountFrequencyEncoder

#from sklearn.cluster import KMeans
#from category_encoders import MEstimateEncoder,OrdinalEncoder
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
#from sklearn.preprocessing import KBinsDiscretizer


#from sklearn.ensemble import StackingRegressor

In [12]:
#DATA LOADING
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()
test.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City
0,0x2318,COIMBRES13DEL01,NaN,NaN,11.003669,76.976494,11.043669,77.016494,30-03-2022,NaN,15:05:00,conditions NaN,NaN,3,Drinks,electric_scooter,1,No,Metropolitian
1,0x3474,BANGRES15DEL01,28,4.6,12.975377,77.696664,13.085377,77.806664,29-03-2022,20:30:00,20:35:00,conditions Windy,Jam,0,Snack,motorcycle,1,No,Metropolitian
2,0x9420,JAPRES09DEL03,23,4.5,26.911378,75.789034,27.001378,75.879034,10-03-2022,19:35:00,19:45:00,conditions Stormy,Jam,0,Drinks,motorcycle,1,No,Metropolitian
3,0x72ee,JAPRES07DEL03,21,4.8,26.766536,75.837333,26.856536,75.927333,02-04-2022,17:15:00,17:20:00,conditions Fog,Medium,1,Meal,scooter,1,No,Metropolitian
4,0xa759,CHENRES19DEL01,31,4.6,12.986047,80.218114,13.096047,80.328114,27-03-2022,18:25:00,18:40:00,conditions Sunny,Medium,2,Drinks,scooter,1,No,Metropolitian


In [13]:
#DATA CLEANING
#test=test.replace(" ","")
test=test.replace('NaN', float(np.nan), regex=True)

#train=train.replace(" ","")
train=train.replace('NaN', float(np.nan), regex=True)

train.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [16]:
train['Weatherconditions']=train['Weatherconditions'].str.split(" ", expand=True)
test['Weatherconditions']=test['Weatherconditions'].str.split(" ", expand=True)

train['Time_taken(min)']=train['Time_taken(min)'].str.split(" ", expand=True)

train.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30


In [17]:
num_cols = ['Delivery_person_Age','Delivery_person_Ratings','Restaurant_latitude','Restaurant_longitude',
            'Delivery_location_latitude','Delivery_location_longitude','Vehicle_condition',
            'multiple_deliveries','Time_taken(min)']
for col in num_cols:
    train[col]=train[col].astype('float64')

for col in num_cols[:-1]:
    test[col]=test[col].astype('float64')

train['Order_Date']=pd.to_datetime(train['Order_Date'],format="%d-%m-%Y")
test['Order_Date']=pd.to_datetime(test['Order_Date'],format="%d-%m-%Y")

In [18]:
train['Time_Orderd']=pd.to_timedelta(train['Time_Orderd'])
train['Time_Order_picked']=pd.to_timedelta(train['Time_Order_picked'])

train['Time_Order_picked_formatted']=np.where(train['Time_Order_picked'] < train['Time_Orderd'], train['Order_Date'] + pd.DateOffset(1)+train['Time_Order_picked'], train['Order_Date']+train['Time_Order_picked'])
train['Time_Ordered_formatted'] = train['Order_Date']+ train['Time_Orderd']
train['order_prepare_time_diff_mins']=((train['Time_Order_picked_formatted']- train['Time_Ordered_formatted']).dt.total_seconds())/60

In [19]:
train.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),Time_Order_picked_formatted,Time_Ordered_formatted,order_prepare_time_diff_mins
0,0x4607,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,0 days 11:30:00,...,2.0,Snack,motorcycle,0.0,No,Urban,24.0,2022-03-19 11:45:00,2022-03-19 11:30:00,15.0
1,0xb379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,0 days 19:45:00,...,2.0,Snack,scooter,1.0,No,Metropolitian,33.0,2022-03-25 19:50:00,2022-03-25 19:45:00,5.0
2,0x5d6d,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,0 days 08:30:00,...,0.0,Drinks,motorcycle,1.0,No,Urban,26.0,2022-03-19 08:45:00,2022-03-19 08:30:00,15.0
3,0x7a6a,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,0 days 18:00:00,...,0.0,Buffet,motorcycle,1.0,No,Metropolitian,21.0,2022-04-05 18:10:00,2022-04-05 18:00:00,10.0
4,0x70a2,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,0 days 13:30:00,...,1.0,Snack,scooter,1.0,No,Metropolitian,30.0,2022-03-26 13:45:00,2022-03-26 13:30:00,15.0


In [20]:
test['Time_Orderd']=pd.to_timedelta(test['Time_Orderd'])
test['Time_Order_picked']=pd.to_timedelta(test['Time_Order_picked'])

test['Time_Order_picked_formatted']=np.where(test['Time_Order_picked'] < test['Time_Orderd'], test['Order_Date'] + pd.DateOffset(1)+test['Time_Order_picked'], test['Order_Date']+test['Time_Order_picked'])
test['Time_Ordered_formatted'] = test['Order_Date']+ test['Time_Orderd']
test['order_prepare_time_diff_mins']=((test['Time_Order_picked_formatted']- test['Time_Ordered_formatted']).dt.total_seconds())/60

In [21]:
cols=['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude']
for col in cols:
    train[col]= abs(train[col])
for col in cols:
    test[col]= abs(test[col])

In [22]:
#Creating new features on the basis of latitude and longitude
train['distance_diff_KM']=np.zeros(len(train))
restaurant_cordinates_train=train[['Restaurant_latitude','Restaurant_longitude']].to_numpy()
delivery_location_cordinates_train=train[['Delivery_location_latitude','Delivery_location_longitude']].to_numpy()

for i in range(len(train)):
    train['distance_diff_KM'].loc[i]=geodesic(restaurant_cordinates_train[i],delivery_location_cordinates_train[i])

In [23]:
test['distance_diff_KM']=np.zeros(len(test))
restaurant_cordinates_test=test[['Restaurant_latitude','Restaurant_longitude']].to_numpy()
delivery_location_cordinates_test=test[['Delivery_location_latitude','Delivery_location_longitude']].to_numpy()

for i in range(len(test)):
    test['distance_diff_KM'].loc[i]=geodesic(restaurant_cordinates_test[i],delivery_location_cordinates_test[i])

In [24]:
y=train["Time_taken(min)"]
train = train.drop(columns=["Time_taken(min)",'ID'],axis=1)
Id= test['ID'].str.strip()
test = test.drop(columns=["ID"],axis=1)

In [26]:
data=pd.concat([train,test]).reset_index(drop=True)
data

,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,...,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_Order_picked_formatted,Time_Ordered_formatted,order_prepare_time_diff_mins,distance_diff_KM
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,0 days 11:30:00,0 days 11:45:00,...,2.0,Snack,motorcycle,0.0,No,Urban,2022-03-19 11:45:00,2022-03-19 11:30:00,15.0,3.0207366434610647 km
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,0 days 19:45:00,0 days 19:50:00,...,2.0,Snack,scooter,1.0,No,Metropolitian,2022-03-25 19:50:00,2022-03-25 19:45:00,5.0,20.143736910348597 km
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,0 days 08:30:00,0 days 08:45:00,...,0.0,Drinks,motorcycle,1.0,No,Urban,2022-03-19 08:45:00,2022-03-19 08:30:00,15.0,1.549692932239404 km
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,0 days 18:00:00,0 days 18:10:00,...,0.0,Buffet,motorcycle,1.0,No,Metropolitian,2022-04-05 18:10:00,2022-04-05 18:00:00,10.0,7.774496620087613 km
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,0 days 13:30:00,0 days 13:45:00,...,1.0,Snack,scooter,1.0,No,Metropolitian,2022-03-26 13:45:00,2022-03-26 13:30:00,15.0,6.197897916874956 km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56987,JAPRES01DEL01,35.0,4.6,26.905190,75.810753,27.015190,75.920753,2022-03-27,0 days 21:35:00,0 days 21:45:00,...,1.0,Snack,scooter,1.0,No,Metropolitian,2022-03-27 21:45:00,2022-03-27 21:35:00,10.0,16.365894271420444 km
56988,JAPRES11DEL01,33.0,4.9,26.902940,75.793007,26.912940,75.803007,2022-03-11,0 days 11:40:00,0 days 11:45:00,...,1.0,Drinks,scooter,1.0,No,Metropolitian,2022-03-11 11:45:00,2022-03-11 11:40:00,5.0,1.4881081439039618 km
56989,SURRES11DEL03,NaN,NaN,21.157735,72.768778,21.217735,72.828778,2022-03-11,NaT,0 days 21:05:00,...,3.0,Drinks,scooter,1.0,No,Metropolitian,2022-03-11 21:05:00,NaT,NaN,9.107658657262554 km
56990,VADRES03DEL02,27.0,4.7,22.320000,73.170000,22.450000,73.300000,2022-03-06,0 days 18:35:00,0 days 18:40:00,...,0.0,Meal,motorcycle,0.0,No,Metropolitian,2022-03-06 18:40:00,2022-03-06 18:35:00,5.0,19.65858405722505 km


In [27]:
na_cols=[]
for col in data.columns:
    if data[col].isna().sum()>0:
        na_cols.append(col)

data["n_missing"]= data[na_cols].isna().sum(axis=1)
data_missing_tag_df = data[na_cols].isna().astype(np.int8)
data_missing_tag_df.columns = [f"{c}_missing" for c in data_missing_tag_df.columns]

data=pd.concat([data, data_missing_tag_df], axis=1)

In [29]:
#Creating time based features
data["day"] = data.Order_Date.dt.day
data["week"] = data.Order_Date.dt.isocalendar().week
data["month"] = data.Order_Date.dt.month
data["quarter"] = data.Order_Date.dt.quarter
data["year"] = data.Order_Date.dt.year
data["hour"] = data.Time_Orderd.dt.components['hours']
data["dayofyear"] = data.Order_Date.dt.dayofyear
data['day_of_week'] = data.Order_Date.dt.day_of_week.astype(int)
data["is_month_start"] = data.Order_Date.dt.is_month_start.astype(int)
data["is_month_end"] = data.Order_Date.dt.is_month_end.astype(int)
data["is_quarter_start"] = data.Order_Date.dt.is_quarter_start.astype(int)
data["is_quarter_end"] = data.Order_Date.dt.is_quarter_end.astype(int)
data["is_year_start"] = data.Order_Date.dt.is_year_start.astype(int)
data["is_year_end"] = data.Order_Date.dt.is_year_end.astype(int)
data["is_leap_year"] = data.Order_Date.dt.is_leap_year.astype(int)
data["days_in_month"] = data.Order_Date.dt.days_in_month
data['is_weekend'] = np.where(data['day_of_week'].isin([5,6]),1,0)

In [30]:
data['distance_diff_KM']=data['distance_diff_KM'].astype("str").str.extract('(\d+)')
data['distance_diff_KM']=data['distance_diff_KM'].astype("int64")

In [31]:
data=data.drop(columns=['Order_Date','Time_Orderd','Time_Order_picked'],axis=1)
data=data.drop(columns=['Time_Order_picked_formatted','Time_Ordered_formatted'])

In [32]:
data['week']=data['week'].astype("int64")

In [33]:
for col in data.columns:
    if data[col].dtype == 'uint8':
        data[col]=data[col].astype("int64")

In [34]:
data['city_code']=data['Delivery_person_ID'].str.split("RES", expand=True)[0]

In [35]:
data=data.drop(columns=["Delivery_person_ID" ],  axis=1)

In [36]:
#Removing constant and duplicate features
pipe = Pipeline([
   ('constant', DropConstantFeatures(tol=0.998,missing_values='ignore')),
   ('duplicated', DropDuplicateFeatures(missing_values='ignore')),
])
data=pipe.fit_transform(data)

NameError: name 'DropConstantFeatures' is not defined

In [37]:
#More Feature Engineering
num_ft = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
mf=MathFeatures(['Delivery_person_Age','Delivery_person_Ratings'],missing_values='ignore',func= ["std",'mean'])
data=mf.fit_transform(data)

NameError: name 'MathFeatures' is not defined

In [38]:
feat1=data.groupby("Delivery_person_Age")["Delivery_person_Ratings"].transform("mean")
data['feat2']=data['Delivery_person_Ratings'] - feat1

feat3=data.groupby("Delivery_person_Age")["Delivery_person_Ratings"].transform("std")
data['feat4']=data['Delivery_person_Ratings'] - feat3

data['feat9']=data.groupby(["Weatherconditions",'Road_traffic_density'])["Delivery_person_Age"].transform("mean")
#data['feat10']=data['Delivery_person_Age'] - data['feat9']

feat11=data.groupby(["Weatherconditions",'Road_traffic_density'])["Delivery_person_Age"].transform("std")
data['feat12']=data['Delivery_person_Age'] - feat11

data['feat13']=data.groupby(["Weatherconditions",'Road_traffic_density','City','city_code'])["distance_diff_KM"].transform("std")
data['feat14']=data['Delivery_person_Age'] - data['feat13']

In [39]:
#Target encoding
encoder = MEstimateEncoder(m=1)

train= data[:len(train)]
test=data[len(train):]

train =  encoder.fit_transform(train,y)
test= encoder.transform(test)

NameError: name 'MEstimateEncoder' is not defined

In [40]:
xgb_model=xgb.XGBRegressor(n_estimators=25,max_depth=7)
lgb = LGBMRegressor(num_leaves=85,max_depth=8,learning_rate=0.1)
cat= CatBoostRegressor(n_estimators=700,max_depth=8,learning_rate=0.05)

NameError: name 'CatBoostRegressor' is not defined

In [41]:
#Feature Selection
selected_feat_=['Delivery_person_Age',
 'Delivery_person_Ratings',
 'Weatherconditions',
 'Road_traffic_density',
 'Vehicle_condition',
 'multiple_deliveries',
 'Festival',
 'City',
 'distance_diff_KM',
 'n_missing',
 'Weatherconditions_missing',
 'multiple_deliveries_missing',
 'Festival_missing',
 'City_missing',
 'is_month_end',
 'is_quarter_start',
 'is_weekend',
 'feat2',
 'feat9',
 'feat12']

In [42]:
imp_feat=selected_feat_[:12]

In [43]:
#Stacking
estimators = [
     ('xgb_model', xgb_model),
     ('lgb', lgb),('cat',cat)]
# best
reg = StackingRegressor(
estimators=estimators,
 final_estimator=xgb_model)

xgb_model.fit(train[imp_feat],y)
reg.fit(train,y)
lgb.fit(train[imp_feat],y)
cat.fit(train[imp_feat],y)

reg_pred=reg.predict(test)
lgb_pred=lgb.predict(test[imp_feat])
cat_pred=cat.predict(test[imp_feat])
xgb_pred=xgb_model.predict(test[imp_feat])

ensemble=reg_pred*0.2 + lgb_pred*0.45 + cat_pred*0.35  # Using these ratios after lots of experimenting

NameError: name 'cat' is not defined